In [1]:
from gensim.corpora import Dictionary
from gensim import matutils
from gensim.models import LdaModel
from gensim.utils import tokenize

from importlib import reload

import logging

import numpy as np

import wiki

logging.basicConfig(
    format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO
)

Using TensorFlow backend.


In [8]:
reload(wiki)

<module 'wiki' from '/Users/theodoreando/Dropbox (Personal)/JPAM-Taxonomy/wiki.py'>

In [2]:
model = LdaModel.load("lda_models/lda2194_lem_100topics_100pass")
model.id2word = Dictionary.load("./2194lem.dict")
topic_term = model.state.get_lambda()

2017-09-01 14:38:05,355 : INFO : loading LdaModel object from lda_models/lda2194_lem_100topics_100pass
2017-09-01 14:38:05,420 : INFO : loading id2word recursively from lda_models/lda2194_lem_100topics_100pass.id2word.* with mmap=None
2017-09-01 14:38:05,421 : INFO : setting ignored attribute state to None
2017-09-01 14:38:05,422 : INFO : setting ignored attribute dispatcher to None
2017-09-01 14:38:05,424 : INFO : loaded lda_models/lda2194_lem_100topics_100pass
2017-09-01 14:38:05,426 : WARNING : random_state not set so using default value
2017-09-01 14:38:05,427 : INFO : loading LdaModel object from lda_models/lda2194_lem_100topics_100pass.state
2017-09-01 14:38:05,478 : INFO : loaded lda_models/lda2194_lem_100topics_100pass.state
2017-09-01 14:38:05,494 : INFO : loading Dictionary object from ./2194lem.dict
2017-09-01 14:38:05,513 : INFO : loaded ./2194lem.dict


# User feedback

In [9]:
def merge_topics(model, t1, t2):
    t1, t2 = sorted([t1, t2])
    print(model.print_topic(t1))
    print(model.print_topic(t2))
    
    model.num_topics -= 1
    state = model.state
    avg_alpha = (model.alpha[t1] + model.alpha[t2]) / 2
    avg_expElogbeta = np.average([model.expElogbeta[t1], model.expElogbeta[t2]])
    avg_eta = np.average([model.eta[t1], model.eta[t2]])
    avg_sstats = np.average([state.sstats[t1], state.sstats[t2]])
    print("Averages:")
    print("\talpha " + repr(avg_alpha))
    print("\texpElogbeta " + repr(avg_expElogbeta))
    print("\teta " + repr(avg_eta))
    print("\tsstats " + repr(avg_sstats))
    
    model.alpha[t2] = avg_alpha
    model.alpha = np.delete(model.alpha, t1, 0)
    
    model.expElogbeta[t2] = avg_expElogbeta
    model.expElogbeta = np.delete(model.expElogbeta, t1, 0)
    
    model.eta[t2] = avg_eta
    model.eta = np.delete(model.eta, t1, 0)
    
    state.eta[t2] = avg_eta
    state.eta = np.delete(state.eta, t1, 0)
    
    state.sstats[t2] = avg_sstats
    state.sstats = np.delete(state.sstats, t1, 0)
    
merge_topics(model, 0, 1)

0.042*"safety" + 0.035*"accident" + 0.031*"fatality" + 0.029*"vehicle" + 0.029*"driver" + 0.020*"seat" + 0.017*"traffic" + 0.017*"alcohol" + 0.017*"belt" + 0.016*"crash"
0.063*"firm" + 0.023*"zone" + 0.023*"industry" + 0.016*"manufacturing" + 0.016*"trade" + 0.015*"enterprise" + 0.015*"business" + 0.014*"hire" + 0.012*"extension" + 0.011*"industrial"
Averages:
	alpha 0.01
	expElogbeta 4.0187683053819465e-05
	eta 0.01
	sstats 1.6787359398863926


In [10]:
text = """Cooperative policies hold out promise of an improvement over 
coercive mandates as ways to enhance implementation of intergovernmental programs."""

doc = tokenize(text, lower=True)
bow = model.id2word.doc2bow(doc)
model.get_document_topics(bow)

[(7, 0.23816940719694354),
 (10, 0.1249467280873473),
 (12, 0.1053469361068716),
 (66, 0.43644183351374422)]

In [11]:
model.update([bow])

2017-08-30 16:46:29,138 : INFO : running online (multi-pass) LDA training, 99 topics, 100 passes over the supplied corpus of 1 documents, updating model once every 1 documents, evaluating perplexity every 1 documents, iterating 50x with a convergence threshold of 0.001000
/usr/local/lib/python3.6/site-packages/gensim/models/ldamodel.py:529: RuntimeWarning: overflow encountered in exp2
  (perwordbound, np.exp2(-perwordbound), len(chunk), corpus_words))
2017-08-30 16:46:29,772 : INFO : -90538.232 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:46:29,774 : INFO : PROGRESS: pass 0, at document #1/1
2017-08-30 16:46:29,835 : INFO : merging changes from 1 documents into a model of 2195 documents
2017-08-30 16:46:30,698 : INFO : topic #90 (0.010): 0.053*"network" + 0.034*"telephone" + 0.030*"company" + 0.022*"homeless" + 0.020*"phone" + 0.019*"homelessness" + 0.018*"fcc" + 0.017*"shelter" + 0.017*"distance" + 0.017*"city"
2017-08-30

2017-08-30 16:46:37,012 : INFO : PROGRESS: pass 5, at document #1/1
2017-08-30 16:46:37,077 : INFO : merging changes from 1 documents into a model of 2195 documents
2017-08-30 16:46:37,778 : INFO : topic #62 (0.010): 0.059*"student" + 0.054*"voucher" + 0.049*"charter" + 0.038*"parent" + 0.038*"educational" + 0.026*"lottery" + 0.017*"attend" + 0.014*"attendance" + 0.011*"religious" + 0.010*"parental"
2017-08-30 16:46:37,782 : INFO : topic #97 (0.010): 0.086*"contract" + 0.026*"competition" + 0.024*"contractor" + 0.023*"privatization" + 0.017*"delivery" + 0.015*"production" + 0.014*"sector" + 0.014*"firm" + 0.014*"transaction" + 0.013*"provision"
2017-08-30 16:46:37,786 : INFO : topic #73 (0.010): 0.031*"forest" + 0.028*"land" + 0.019*"property" + 0.016*"park" + 0.013*"species" + 0.010*"water" + 0.010*"protection" + 0.010*"card" + 0.009*"basin" + 0.008*"conservation"
2017-08-30 16:46:37,790 : INFO : topic #35 (0.010): 0.042*"hiv" + 0.041*"blood" + 0.028*"aids" + 0.027*"testing" + 0.018*"

2017-08-30 16:46:44,265 : INFO : topic #2 (0.010): 0.012*"sanction" + 0.012*"company" + 0.008*"netherlands" + 0.007*"chapter" + 0.007*"eitc" + 0.006*"city" + 0.006*"tool" + 0.005*"regime" + 0.005*"instrument" + 0.005*"germany"
2017-08-30 16:46:44,267 : INFO : topic #21 (0.010): 0.127*"manhattan" + 0.027*"backer" + 0.017*"overestimating" + 0.011*"whitehouse" + 0.003*"ballistic" + 0.000*"gal" + 0.000*"repertoire" + 0.000*"gao" + 0.000*"unnecessarily" + 0.000*"gag"
2017-08-30 16:46:44,269 : INFO : topic #93 (0.010): 0.104*"policv" + 0.076*"nathan" + 0.049*"erica" + 0.040*"nsw" + 0.019*"poorhouse" + 0.000*"gal" + 0.000*"gag" + 0.000*"repertoire" + 0.000*"socio" + 0.000*"schultze"
2017-08-30 16:46:44,295 : INFO : topic diff=0.019138, rho=0.021291
2017-08-30 16:46:44,819 : INFO : -88064.832 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:46:44,822 : INFO : PROGRESS: pass 11, at document #1/1
2017-08-30 16:46:44,883 : INFO : merging

2017-08-30 16:46:52,027 : INFO : topic diff=0.019078, rho=0.021267
2017-08-30 16:46:52,614 : INFO : -86936.124 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:46:52,616 : INFO : PROGRESS: pass 16, at document #1/1
2017-08-30 16:46:52,670 : INFO : merging changes from 1 documents into a model of 2195 documents
2017-08-30 16:46:53,647 : INFO : topic #42 (0.010): 0.060*"court" + 0.020*"admission" + 0.019*"civil" + 0.017*"legal" + 0.016*"judicial" + 0.013*"title" + 0.011*"race" + 0.010*"litigation" + 0.010*"selective" + 0.010*"judge"
2017-08-30 16:46:53,650 : INFO : topic #27 (0.010): 0.203*"life" + 0.092*"injury" + 0.068*"helmet" + 0.063*"meta" + 0.053*"vsl" + 0.050*"death" + 0.040*"juvenile" + 0.034*"pedestrian" + 0.033*"legislation" + 0.032*"fatality"
2017-08-30 16:46:53,652 : INFO : topic #8 (0.010): 0.048*"hospital" + 0.045*"medical" + 0.029*"patient" + 0.014*"physician" + 0.014*"provider" + 0.011*"payment" + 0.011*"home" + 

2017-08-30 16:47:02,230 : INFO : topic #81 (0.010): 0.083*"father" + 0.045*"parent" + 0.022*"enforcement" + 0.021*"noncustodial" + 0.018*"payment" + 0.017*"man" + 0.016*"guideline" + 0.014*"paternity" + 0.014*"custodial" + 0.012*"eitc"
2017-08-30 16:47:02,234 : INFO : topic #47 (0.010): 0.575*"california" + 0.141*"berkeley" + 0.025*"rivlin" + 0.018*"arnold" + 0.017*"meltsner" + 0.015*"schelling" + 0.009*"brandl" + 0.009*"ave" + 0.009*"rochester" + 0.009*"blomquist"
2017-08-30 16:47:02,238 : INFO : topic #85 (0.010): 0.199*"tax" + 0.047*"taxe" + 0.015*"sale" + 0.013*"revenue" + 0.013*"taxpayer" + 0.012*"taxation" + 0.011*"windfall" + 0.011*"personal" + 0.011*"burden" + 0.009*"household"
2017-08-30 16:47:02,256 : INFO : topic diff=0.019005, rho=0.021238
2017-08-30 16:47:02,828 : INFO : -85576.898 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:47:02,831 : INFO : PROGRESS: pass 22, at document #1/1
2017-08-30 16:47:02,889 : INFO

...

2017-08-30 16:48:13,245 : INFO : topic #95 (0.010): 0.014*"nonprofit" + 0.014*"field" + 0.010*"professional" + 0.009*"poor" + 0.006*"woman" + 0.006*"network" + 0.006*"accountability" + 0.006*"funding" + 0.006*"regime" + 0.005*"citizen"
2017-08-30 16:48:13,276 : INFO : topic diff=0.018443, rho=0.020984
2017-08-30 16:48:13,833 : INFO : -73018.303 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:48:13,835 : INFO : PROGRESS: pass 76, at document #1/1
2017-08-30 16:48:13,898 : INFO : merging changes from 1 documents into a model of 2195 documents
2017-08-30 16:48:14,761 : INFO : topic #7 (0.010): 0.112*"cooperative" + 0.044*"implementation" + 0.030*"regulation" + 0.023*"regulatory" + 0.023*"compliance" + 0.022*"enforcement" + 0.019*"enhance" + 0.016*"mandate" + 0.011*"inspection" + 0.007*"commitment"
2017-08-30 16:48:14,764 : INFO : topic #47 (0.010): 0.483*"california" + 0.118*"berkeley" + 0.021*"rivlin" + 0.015*"arnold" + 0.014*"

2017-08-30 16:48:22,319 : INFO : merging changes from 1 documents into a model of 2195 documents
2017-08-30 16:48:23,173 : INFO : topic #28 (0.010): 0.076*"kindergarten" + 0.021*"fdi" + 0.019*"childcare" + 0.018*"deliberation" + 0.016*"dsm" + 0.016*"cutler" + 0.015*"candidate" + 0.014*"entrance" + 0.013*"day" + 0.013*"globalization"
2017-08-30 16:48:23,176 : INFO : topic #47 (0.010): 0.471*"california" + 0.115*"berkeley" + 0.020*"rivlin" + 0.014*"arnold" + 0.014*"meltsner" + 0.013*"schelling" + 0.007*"brandl" + 0.007*"ave" + 0.007*"rochester" + 0.007*"blomquist"
2017-08-30 16:48:23,180 : INFO : topic #89 (0.010): 0.070*"employment" + 0.060*"wage" + 0.053*"worker" + 0.045*"earning" + 0.014*"hour" + 0.013*"employer" + 0.012*"minimum" + 0.010*"skill" + 0.009*"working" + 0.009*"supply"
2017-08-30 16:48:23,181 : INFO : topic #22 (0.010): 0.026*"congress" + 0.017*"committee" + 0.012*"congressional" + 0.009*"house" + 0.009*"legislation" + 0.008*"proposal" + 0.007*"bill" + 0.006*"president" + 

2017-08-30 16:48:30,954 : INFO : topic #84 (0.010): 0.031*"graduate" + 0.030*"affair" + 0.015*"sematech" + 0.012*"international" + 0.012*"pennsylvania" + 0.009*"consortium" + 0.008*"college" + 0.008*"doctoral" + 0.008*"essay" + 0.007*"george"
2017-08-30 16:48:30,960 : INFO : topic #5 (0.010): 0.059*"ay" + 0.034*"dcps" + 0.021*"ppp" + 0.021*"toyota" + 0.019*"cruz" + 0.016*"biomass" + 0.016*"threat" + 0.004*"rudiger" + 0.004*"rhee" + 0.002*"competitiveness"
2017-08-30 16:48:30,963 : INFO : topic #62 (0.010): 0.057*"student" + 0.053*"voucher" + 0.047*"charter" + 0.037*"parent" + 0.036*"educational" + 0.025*"lottery" + 0.016*"attend" + 0.013*"attendance" + 0.011*"religious" + 0.009*"parental"
2017-08-30 16:48:30,980 : INFO : topic diff=0.018373, rho=0.020934
2017-08-30 16:48:31,530 : INFO : -70373.426 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:48:31,533 : INFO : PROGRESS: pass 87, at document #1/1
2017-08-30 16:48:31,584 : I

2017-08-30 16:48:38,675 : INFO : topic diff=0.018350, rho=0.020911
2017-08-30 16:48:39,229 : INFO : -69160.980 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:48:39,232 : INFO : PROGRESS: pass 92, at document #1/1
2017-08-30 16:48:39,281 : INFO : merging changes from 1 documents into a model of 2195 documents
2017-08-30 16:48:40,278 : INFO : topic #32 (0.010): 0.146*"poverty" + 0.054*"poor" + 0.018*"inequality" + 0.017*"household" + 0.010*"distribution" + 0.010*"head" + 0.010*"census" + 0.009*"line" + 0.009*"life" + 0.008*"adult"
2017-08-30 16:48:40,282 : INFO : topic #1 (0.010): 0.070*"cell" + 0.035*"stem" + 0.023*"party" + 0.021*"soft" + 0.021*"hesc" + 0.016*"funding" + 0.016*"ssb" + 0.014*"line" + 0.014*"soda" + 0.014*"bcra"
2017-08-30 16:48:40,284 : INFO : topic #66 (0.010): 0.083*"out" + 0.082*"promise" + 0.082*"to" + 0.072*"enhance" + 0.063*"implementation" + 0.008*"manager" + 0.008*"city" + 0.007*"staff" + 0.005*"mpp" 

2017-08-30 16:48:48,513 : INFO : topic #91 (0.010): 0.034*"military" + 0.034*"fire" + 0.025*"soviet" + 0.018*"east" + 0.018*"personnel" + 0.014*"epstein" + 0.011*"function" + 0.011*"european" + 0.010*"africa" + 0.009*"output"
2017-08-30 16:48:48,514 : INFO : topic #40 (0.010): 0.051*"variable" + 0.022*"sample" + 0.016*"coefficient" + 0.014*"regression" + 0.011*"equation" + 0.011*"error" + 0.010*"pam" + 0.009*"column" + 0.009*"behalf" + 0.009*"percentage"
2017-08-30 16:48:48,516 : INFO : topic #46 (0.010): 0.014*"corruption" + 0.012*"partisan" + 0.012*"student" + 0.010*"monetary" + 0.010*"politic" + 0.009*"inflation" + 0.009*"gender" + 0.008*"woman" + 0.008*"re" + 0.008*"kirp"
2017-08-30 16:48:48,535 : INFO : topic diff=0.018330, rho=0.020883
2017-08-30 16:48:49,103 : INFO : -67697.827 per-word bound, inf perplexity estimate based on a held-out corpus of 1 documents with 9 words
2017-08-30 16:48:49,106 : INFO : PROGRESS: pass 98, at document #1/1
2017-08-30 16:48:49,165 : INFO : merging

# A\* with LDA inference

In [3]:
page1 = "Nonpoint source water pollution regulations in the United States"
page2 = "Superfund"

In [4]:
%prun wiki.A_star_sca('Decriminalization of non-medical cannabis in the United States', 'Arguments for and against drug prohibition', model)


2017-09-01 14:38:32,954 : INFO : Distance between 'Decriminalization of non-medical cannabis in the United States' and 'Arguments for and against drug prohibition' : -0.882305
2017-09-01 14:38:32,955 : INFO : Distance between 'Arguments for and against drug prohibition' and 'Decriminalization of non-medical cannabis in the United States' : -0.882305
2017-09-01 14:38:32,956 : INFO : Advancing 1 to  Decriminalization of non-medical cannabis in the United States with distance -0.882305
2017-09-01 14:38:32,958 : INFO : Advancing 2 to  Arguments for and against drug prohibition with distance -0.882305
2017-09-01 14:38:51,402 : INFO : Distance between 'Category:Cannabis in the United States' and 'Arguments for and against drug prohibition' : -0.825727
2017-09-01 14:39:04,455 : INFO : Distance between 'Category:Cannabis law in the United States' and 'Arguments for and against drug prohibition' : -0.758108
2017-09-01 14:39:16,565 : INFO : Distance between 'Category:Cannabis law reform in the U

In [5]:
wiki.list_sca([page1], [page2])

Limiter 3; Front1: 1; Front2: 1
Limiter 2; Front1: 3; Front2: 9
Limiter 1; Front1: 9; Front2: 34
{'Category:Pollution in the United States'} 3
{'Category:Pollution in the United States'} 3
{'Category:Pollution in the United States', 'Category:Environmental law in the United States'} 3
Limiter 1; Front1: 26; Front2: 101
{'Category:Pollution in the United States', 'Category:Environmental law in the United States'} 3


{'Category:Environmental law in the United States',
 'Category:Pollution in the United States'}

In [ ]:
import pickle